In [34]:
import torch

class DenseLayer:

  # Layer initialization
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.bias = torch.rand(n_neurons)

  # Forward pass
  def forward(self, inputs):
    # Calculate output values from inputs, weights and biases
    self.output = torch.matmul(inputs, self.weights) + self.bias


class Loss_CategoricalCrossentropy:
    def forward(self, y_pred, y_true):
        # Ensure probabilities do not equal 0
        y_pred_clipped = torch.clamp(y_pred, 1e-7, 1 - 1e-7)

        # If targets are one-hot encoded, convert them to discrete labels
        if len(y_true.shape) == 2:
            y_true = torch.argmax(y_true, dim=1)

        # Get the predicted probabilities for the correct classes
        correct_confidences = y_pred_clipped[torch.arange(len(y_pred)), y_true]

        # Calculate the negative log likelihood
        negative_log_likelihoods = -torch.log(correct_confidences)

        # Calculate the overall loss as the average of individual sample losses
        loss = torch.mean(negative_log_likelihoods)

        return loss



class Accuracy:
    def __init__(self):
        self.output = 0
    def forward(self,y_pred,y_true):
        if y_pred.shape != y_true.shape:
            one_hot_notation = torch.zeros(y_pred.shape)
            one_hot_notation[range(len(y_pred)),y_true] = 1
        else:
            one_hot_notation = y_true
        correct_values = y_pred==one_hot_notation
        correct_values = correct_values * one_hot_notation
        self.output = torch.sum(correct_values) / len(y_pred)
        return self.output


# Sigmoid activation
class Activation_Sigmoid:
  # Forward pass
  def forward(self, inputs):
    self.output = 1 / (1 + torch.exp(-inputs))



# ReLU activation
class Activation_ReLU:
  # Forward pass
  def forward(self, inputs):
    self.output = torch.max(inputs, torch.tensor(0))
    return self.output


# Classification Model
class ClassificationModel:
    def __init__(self,num_of_features:int,num_of_classes:int):
        self.layer1 = DenseLayer(num_of_features,4)
        self.activation1 = Activation_ReLU()
        self.output_layer = DenseLayer(4,num_of_classes)
        self.output_activation = Activation_Sigmoid()
        self.accuracy = Accuracy()
        self.errors = [float('inf')] * num_of_classes

    def forward_propagate(self,inputs):
        self.layer1.forward(inputs)
        self.activation1.forward(self.layer1.output)
        self.output_layer.forward(self.activation1.output)
        self.output_activation.forward(self.output_layer.output)
        self.inputs = inputs

    def loss_and_accuracy(self, target):
        self.true_value = target
        if self.output_activation.output.shape != target.shape:
            one_hot_notation = torch.zeros(self.output_activation.output.shape)
            one_hot_notation[range(len(self.output_activation.output)), target] = 1
            self.true_value = one_hot_notation

        squared_errors = (self.true_value - self.output_layer.output) ** 2
        mean_loss = torch.mean(squared_errors) / 2

        accuracy = self.accuracy.forward(self.output_layer.output, self.true_value)

        return mean_loss, accuracy



    def back_prop(self, lr):
        errors = -(self.true_value - self.output_layer.output)  # d(y-output)/d(output)
        sigmoid_derivative = self.output_activation.output * (1 - self.output_activation.output)
        output_errors = errors * sigmoid_derivative

        # Update weights and biases for the output layer
        self.output_layer.bias -= lr * torch.mean(output_errors, dim=0)
        self.output_layer.weights -= lr * torch.matmul(self.activation1.output.T, output_errors) / len(output_errors)

        # Backpropagate errors to the first layer
        relu_derivative = torch.where(self.activation1.output > 0, torch.tensor(1.0), torch.tensor(0.0))
        hidden_errors = torch.matmul(output_errors, self.output_layer.weights.T) * relu_derivative

        # Update weights and biases for the first layer
        self.layer1.bias -= lr * torch.mean(hidden_errors, dim=0)
        self.layer1.weights -= lr * torch.matmul(self.inputs.T, hidden_errors) / len(hidden_errors)





model = ClassificationModel(2,4)
y = torch.tensor([1,1,0])
x = torch.tensor([[1,2],[3,4],[4,5]],dtype=torch.float)
model.forward_propagate(x)
model.loss_and_accuracy(y)

loss = 0.1
error = float('inf')
iterations = 0
while loss < error:
    iterations += 1
    model.forward_propagate(x)
    model.back_prop(0.01)
    error,acc = model.loss_and_accuracy(y)

print("iterations:",iterations)
print("true vaues:",model.true_value)
print("accuracy:",acc)
print("final output:",model.output_layer.output)
print("final error:",error)


iterations: 115
true vaues: tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]])
accuracy: tensor(0.)
final output: tensor([[0.6733, 0.5895, 0.4999, 0.2269],
        [0.6275, 0.6018, 0.4251, 0.1883],
        [0.6046, 0.6079, 0.3877, 0.1690]])
final error: tensor(0.0999)
